# **model.ipynb**

중간 미션 task는 주어진 입력 문장을 긍정/부정으로 이진분류하는 모델을 만드는 것이다. 이진분류를 위한 모델을 생성하기 위해서, 우리는 사전 훈련된 Bert 모델을 가지고 온 뒤 네이버 영화 리뷰 데이터셋을 활용하여 해당 모델을 긍정/부정의 이진분류를 수행하는 모델로 학습시켜야 한다.

**본 파일에서는 사전 훈련된 Bert 모델을 가지고 와서 우리의 task에 필요한 모델을 생성하고, 모델의 학습 과정에 필요한 초매개변수를 설정하는 과정을 다뤄볼 것이다.**

연습 문제를 시작하기에 앞서, 필요한 라이브러리들을 설치하자.

In [12]:
import torch
from transformers import get_linear_schedule_with_warmup, BertForSequenceClassification, BertConfig

In [13]:
torch.__version__

'1.7.1+cu110'

감성 이진 분류 task를 위한 BertModel을 생성하자.
- 우선, 우리가 생성할 모델을 저장할 PATH를 지정하자.
- 이후, 중간 미션 task가 무엇인지 고려하여 HuggingFace 홈페이지에서 우리에게 필요한 BertModel 형식이 무엇인지 파악하자. 그리고 해당 모델을 이용하여 model을 생성하자. (*본 중간 미션이, 영화리뷰를 긍정과 부정의 두 가지 감정으로 분류해내는 작업임을 고려해보자.)
- 참고로 우리의 모델은 monologg의 kobert을 사전 훈련된 모델로 사용할 것이다. 사전 훈련 모델을 사용하기 위해서 from_pretrained 함수를 사용해야 하며, 함수의 파라미터에 'monologg/kobert'와 라벨 개수(num_labels)를 지정해야 한다.
- 모델을 생성하였다면, 미리 지정해둔 PATH에 해당 모델을 저장한다.

In [16]:
# [분류를 위한 BERT 모델 생성: BertModel을 초기화하는 역할]
def BertModelInitialization():
    # PATH = "/content/gdrive/MyDrive/BERT/model.pt"
    PATH = "model.pt"

    # BertModel은 다양한 작업을 진행할 수 있도록 여러 인터페이스들을 제공한다.
    # 본 중간 미션의 task가 '영화리뷰(Sequence)를 긍정과 부정의 두 가지 감정으로 분류하기(Classification)'이다.
    # Bert에서 시퀀스를 분류하는 인터페이스로 BertForSequenceClassification가 제공되고 있다.
    # 이 외 제공되는 인터페이스 종류가 궁금하다면 HuggingFace 홈페이지에 Bert를 검색해서 찾아보자: https://huggingface.co/docs/transformers/main/en/index
    # 추가로, monologg에 의해 사전훈련된 kobert 모델을 가지고 와야 하므로 .from_pretrained('monologg/kobert')를 쓰고, 분류 라벨 수가 2개 이므로 'num_labels=2'를 추가 입력한다.

    model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2)


    torch.save(model.state_dict(), PATH) # 생성한 모델을 특정 PATH에 저장하기

get_model은 생성한 BertModel을 불러와서, 그것을 우리의 디바이스에 등록하는 함수이다.

In [17]:
def get_model(device):
    PATH = "model.pt"

    model = BertForSequenceClassification.from_pretrained('monologg/kobert')

    model.load_state_dict(torch.load(PATH)) # PATH에 저장된 모델을 불러오기
    model = model.to(device) # 불러온 모델을 device에 올리기

    return model

이번에는, model의 학습 초매개변수(옵티마이저, 에포크, 훈련 스텝, 스케줄러)를 설정하는 **get_model_with_params** 함수를 정의하자.
- 우리는 학습 단계에서 최적의 매개변수를 찾아내기 위하여 옵티마이저를 사용할 수 있다. 대표적인 옵티마이저로 AdamW가 있고, 이 외에도 이하와 같이 다양한 옵티마이저들이 있다.

> Batch Gradient Descent.
Stochastic Gradient Descent.
Momentum. Nesterov Accelerated Gradient
(NAG)
Adagrad.
RMSprop.
Adam.

- 러닝 스케줄러는 학습이 이루어짐에 따라 learning_rate을 감소시키는 도구이다. 러닝 스케줄러 역시 get_linear_schedule_with_warmup 외에 다양한 종류가 있다.

우선 중간미션에서는 AdamW으로 옵티마이저 종류 및 파라미터 내용, 학습 에폭 수, 총 훈련 스텝, 러닝 스케줄러를 모두 고정할 것이다. 최종 미션에서는 이러한 초매개변수를 직접 조절하면서 모델 성능 향상을 위한 다양한 방법을 시도해볼 것이다.

In [18]:
def get_model_with_params(num_data, device):
    model = get_model(device)

    # 옵티마이저 설정하기
    optimizer = torch.optim.AdamW(model.parameters(),
                      lr = 1e-5, # 학습률
                      eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값,
                    )
    # 전체 데이터가 총 몇 번 학습되는지
    epochs = 3

    # 총 훈련 스텝
    total_steps = num_data * epochs

    # 학습이 이루어짐에 따라 learning_rate을 감소시키기 위한 스케줄러
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)

    return model, optimizer, scheduler, epochs


**main** 함수를 통해, 다른 파일에서 import하기 전, model.ipynb 내에 정의된 함수가 성공적으로 구현되어 실행되는지 확인해보자.

In [19]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    BertModelInitialization()
    print(get_model_with_params(200000, device))

In [20]:
if __name__ == '__main__':
      main()

AttributeError: module 'torch' has no attribute 'frombuffer'

- https://pytorch.org/docs/stable/generated/torch.frombuffer.html#torch.frombuffer  
This function’s behavior is undefined when passed an object implementing the buffer protocol  
whose data is not on the CPU. Doing so is likely to cause a segmentation fault.